<a href="https://colab.research.google.com/github/S-Devisri01/Python-colab/blob/main/Text_Generation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install -q tensorflow numpy requests

import numpy as np
import tensorflow as tf
import requests
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# ================================
# 📥 Download Dataset (Tiny Shakespeare)
# ================================
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
text = requests.get(url).text.lower()

print("Total characters:", len(text))
print(text[:500])

# ================================
# 🔤 Create Character Mappings
# ================================
chars = sorted(list(set(text)))
vocab_size = len(chars)

char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

print("Unique characters:", vocab_size)

# ================================
# 🔁 Prepare Sequences
# ================================
SEQ_LEN = 40
STEP = 3

sentences = []
next_chars = []

for i in range(0, len(text) - SEQ_LEN, STEP):
    sentences.append(text[i:i + SEQ_LEN])
    next_chars.append(text[i + SEQ_LEN])

print("Total sequences:", len(sentences))

# ================================
# 📊 Vectorization
# ================================
X = np.zeros((len(sentences), SEQ_LEN, vocab_size), dtype=np.bool_)
y = np.zeros((len(sentences), vocab_size), dtype=np.bool_)

for i, sentence in enumerate(sentences):
    for t, ch in enumerate(sentence):
        X[i, t, char_to_idx[ch]] = 1
    y[i, char_to_idx[next_chars[i]]] = 1

# ================================
# 🧠 Build LSTM Model
# ================================
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LEN, vocab_size)))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001)
)

model.summary()

# ================================
# 🚀 Train Model
# ================================
model.fit(
    X,
    y,
    batch_size=128,
    epochs=20
)

# ================================
# 🔮 Text Generation Function
# ================================
def generate_text(model, seed_text, length=400, temperature=1.0):
    generated = ""
    sentence = seed_text

    for _ in range(length):
        x_pred = np.zeros((1, SEQ_LEN, vocab_size))
        for t, ch in enumerate(sentence):
            x_pred[0, t, char_to_idx[ch]] = 1

        preds = model.predict(x_pred, verbose=0)[0]
        preds = np.log(preds + 1e-8) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)

        next_index = np.random.choice(len(preds), p=preds)
        next_char = idx_to_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

    return generated

# ================================
# 📌 Generate Sample Text
# ================================
start_index = random.randint(0, len(text) - SEQ_LEN - 1)
seed_text = text[start_index:start_index + SEQ_LEN]

print("Seed text:")
print(seed_text)
print("\nGenerated text:\n")

generated_text = generate_text(
    model,
    seed_text,
    length=400,
    temperature=0.8
)

print(generated_text)


Total characters: 1115394
first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you are all resolved rather to die than to famish?

all:
resolved. resolved.

first citizen:
first, you know caius marcius is chief enemy to the people.

all:
we know't, we know't.

first citizen:
let us kill him, and we'll have corn at our own price.
is't a verdict?

all:
no more talking on't; let it be done: away, away!

second citizen:
one word, good citizens.

first citizen:
we are accounted poor
Unique characters: 39
Total sequences: 371785


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        86,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 39)             │         5,031 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 91,047 (355.65 KB)

 Trainable params: 91,047 (355.65 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 285s 97ms/step - loss: 2.5741
Epoch 2/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 283s 98ms/step - loss: 1.9789
Epoch 3/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 282s 97ms/step - loss: 1.8349
Epoch 4/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 286s 98ms/step - loss: 1.7409
Epoch 5/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 321s 98ms/step - loss: 1.6704
Epoch 6/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 322s 98ms/step - loss: 1.6149
Epoch 7/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 282s 97ms/step - loss: 1.5691
Epoch 8/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 281s 97ms/step - loss: 1.5328
Epoch 9/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 282s 97ms/step - loss: 1.5052
Epoch 10/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 284s 98ms/step - loss: 1.4813
Epoch 11/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 283s 97ms/step - loss: 1.4593
Epoch 12/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 280s 96ms/step - loss: 1.4402
Epoch 13/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 281s 97ms/step - loss: 1.4262
Epoch 14/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 280s 96ms/step - 